<a href="https://colab.research.google.com/github/2021-FIIT-Bc-projects/BP-Eliseeva-Najdiskutovanejsie_temy_v_dokumentoch/blob/main/Bakalarka_NIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyLDAvis

In [2]:
import gensim
import nltk
import pandas as pd
import re
import collections
import itertools
import warnings
import pyLDAvis
import pyLDAvis.gensim_models

from gensim import corpora
from sklearn.datasets import fetch_20newsgroups
from wordcloud import WordCloud
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from pprint import pprint

import matplotlib.pyplot as plt
import spacy

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [3]:
import pkg_resources
print(pkg_resources.get_distribution('gensim').version)
print(pkg_resources.get_distribution('nltk').version)
print(pkg_resources.get_distribution('pandas').version)
print(pkg_resources.get_distribution('spacy').version)
print(pkg_resources.get_distribution('pyLDAvis').version)
print(pkg_resources.get_distribution('wordcloud').version)
print(pkg_resources.get_distribution('matplotlib').version)


3.6.0
3.2.5
1.3.4
2.2.4
3.3.1
1.5.0
3.2.2


In [4]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['let', 'fig'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [7]:
def ShowWordCloud(data):
    string = ','.join(str(text) for text in data)
    # Create a WordCloud object
    wordcloud = WordCloud(background_color="white", max_words=5000, max_font_size=60, regexp=r"\w[\w']+\w").generate(string)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()


def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)


def remove_stopwords_spacy(texts):
  filtered_texts = []
  for text in texts:
    filtered_sentence = [] 
    for word in text:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    filtered_texts.append(filtered_sentence)
  return filtered_texts

def remove_stopwords_nltk(texts):
  filtered_texts = []
  for doc in texts:
    filtered_sentence = []
    for word in gensim.utils.simple_preprocess(str(doc)):
      if word not in stop_words:
        filtered_sentence.append(word)
    filtered_texts.append(filtered_sentence)
  return filtered_texts

def twenty_newsgroup_to_list():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    data = {'texts': newsgroups_train.data, 'topic_names': newsgroups_train['target_names'], 'topics_assigned': newsgroups_train['target']}
    return data

def remove_twosymbols(texts):
  for text in texts:
    for word in text:
      if len(word) <= 2:
        text.remove(word)
  return texts

def prepare_data(data, threshold):
  data_words = list(sent_to_words(data))

  data_words_nostops = remove_stopwords_nltk(data_words)

  data_words_nostops_shortRemoved = remove_twosymbols(data_words_nostops)

  bigram = gensim.models.Phrases(data_words_nostops_shortRemoved, threshold=threshold)
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  data_bigrams = [bigram_mod[text] for text in data_words_nostops_shortRemoved]

  data_lemmatized = []
  for text in data_bigrams:
    doc = nlp(" ".join(text))
    data_lemmatized.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
  
  return data_lemmatized

def clean_data(papers):
  papers = papers.drop(columns=['id', 'event_type', 'pdf_name', 'year', 'abstract'], axis=1)
  papers['paper_text_processed'] = papers['paper_text'].str.replace('[ \t\n\r\f\v]', " ")
  papers['paper_text_processed'] = papers['paper_text_processed'].str.replace(r"\S*@\S*\s?", "")
  papers['paper_text_processed'] = papers['paper_text_processed'].str.lower()
  return papers

In [8]:
papers = pd.read_csv('drive/MyDrive/Colab Notebooks/papers_0-1000.csv')
papers = clean_data(papers)
data_lemmatized = prepare_data(papers['paper_text_processed'].values, 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


In [10]:
papers_test = pd.read_csv('drive/MyDrive/Colab Notebooks/papers_5000.csv')
papers_test = clean_data(papers_test)
data_test_lemmatized = prepare_data(papers_test['paper_text_processed'].values, 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


In [12]:
papers_all = pd.read_csv('drive/MyDrive/Colab Notebooks/papers.csv')
papers_all = clean_data(papers_all)
data_all_lemmatized = prepare_data(papers_all['paper_text_processed'].values, 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


In [16]:
id2word = corpora.Dictionary(data_lemmatized)

# print(len(id2word))
id2word.filter_extremes(no_above=0.5, no_below=5)
# print(len(id2word))

corpus_train = [id2word.doc2bow(text) for text in data_lemmatized]
corpus_test = [id2word.doc2bow(text) for text in data_test_lemmatized]
corpus_all = [id2word.doc2bow(text) for text in data_all_lemmatized]

In [34]:
def measure_model(model):
  #perplexity
  print(model.log_perplexity(corpus_test))
  #coherence
  coherence_model_lda = CoherenceModel(model=model, texts=data_test_lemmatized, dictionary=id2word, coherence='c_v')
  print(coherence_model_lda.get_coherence())

In [32]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)
pprint(lda_model.print_topics())

[(0,
  '0.141*"node" + 0.128*"tree" + 0.046*"expert" + 0.044*"level" + '
  '0.037*"graph" + 0.030*"parent" + 0.023*"hierarchical" + 0.021*"structure" + '
  '0.020*"prune" + 0.019*"partition"'),
 (1,
  '0.025*"gaussian" + 0.024*"prior" + 0.024*"density" + 0.021*"mixture" + '
  '0.020*"sample" + 0.018*"posterior" + 0.015*"inference" + '
  '0.013*"approximation" + 0.013*"conditional" + 0.010*"approximate"'),
 (2,
  '0.039*"neuron" + 0.035*"cell" + 0.020*"activity" + 0.019*"response" + '
  '0.016*"stimulus" + 0.013*"visual" + 0.013*"spatial" + 0.013*"connection" + '
  '0.011*"synaptic" + 0.010*"cortical"'),
 (3,
  '0.101*"code" + 0.043*"binary" + 0.037*"bit" + 0.032*"temperature" + '
  '0.029*"noise" + 0.019*"particle" + 0.018*"neuron" + 0.014*"computational" + '
  '0.013*"noisy" + 0.013*"threshold"'),
 (4,
  '0.029*"cluster" + 0.019*"dimensional" + 0.018*"distance" + 0.017*"map" + '
  '0.015*"basis" + 0.014*"structure" + 0.013*"component" + 0.011*"transform" + '
  '0.010*"transformation" 

In [35]:
measure_model(lda_model)

-7.339451858634225
0.4906613940935804


In [33]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_all, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.194254 -0.010261       1        1  14.797046
6     -0.036904 -0.000917       2        1  13.016556
1     -0.164378 -0.020972       3        1  10.055021
17    -0.147189 -0.016237       4        1   8.119971
4     -0.154356 -0.045479       5        1   7.606719
9     -0.147898  0.019247       6        1   7.432719
13     0.056509 -0.098082       7        1   6.411981
2      0.146776 -0.113585       8        1   4.119868
18    -0.115841 -0.085113       9        1   3.551061
19     0.020104 -0.094513      10        1   3.093490
8      0.157870 -0.144129      11        1   3.029561
15     0.008411  0.044480      12        1   2.918806
14    -0.075239 -0.012450      13        1   2.783738
16     0.103433  0.028295      14        1   2.566709
5      0.023796  0.146793      15        1   2.333453
10     0.097131  0.091546      16        1   2.194505
12     0.086239 -0.093181      17        1   1.848503
0      0.057941  0.365160      18        1   1.788180
11     0.228663 -0.037396      19        1   1.324420
3      0.049186  0.076794      20        1   1.007693, topic_info=          Term          Freq         Total Category  logprob  loglift
212      image  39651.000000  39651.000000  Default  30.0000  30.0000
294       node  18032.000000  18032.000000  Default  29.0000  29.0000
454       tree  14978.000000  14978.000000  Default  28.0000  28.0000
926       hide  17476.000000  17476.000000  Default  27.0000  27.0000
586      layer  14812.000000  14812.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
258       loop    338.909450   1479.824687  Topic20  -5.2664   3.1236
336    product    467.100487   4772.683662  Topic20  -4.9456   2.2734
2156    encode    449.828445   4342.773285  Topic20  -4.9833   2.3301
231   interval    367.892204   2752.968598  Topic20  -5.1844   2.5849
145    element    348.390260   9339.582781  Topic20  -5.2389   1.3088

[1351 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5547      1  0.011627  abbreviation
5547      2  0.011627  abbreviation
5547      3  0.011627  abbreviation
5547      4  0.011627  abbreviation
5547      5  0.011627  abbreviation
...     ...       ...           ...
5115      4  0.985789            yk
5115      5  0.001751            yk
5115      6  0.001751            yk
5115     16  0.001751            yk
5115     18  0.001751            yk

[11763 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 2, 18, 5, 10, 14, 3, 19, 20, 9, 16, 15, 17, 6, 11, 13, 1, 12, 4])

In [22]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [24]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 

In [25]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_train, num_topics=20, id2word=id2word)
pprint(ldamallet.show_topics(formatted=False))

[(19,
  [('class', 0.055237304552373044),
   ('classification', 0.04884461048844611),
   ('classifier', 0.03688944236889442),
   ('sample', 0.02017434620174346),
   ('table', 0.016936488169364883),
   ('selection', 0.01690881416908814),
   ('label', 0.015193026151930262),
   ('decision', 0.014943960149439602),
   ('cost', 0.01145703611457036),
   ('technique', 0.010875882108758821)]),
 (10,
  [('circuit', 0.03975704030922143),
   ('chip', 0.02834529725750046),
   ('voltage', 0.027351371249769923),
   ('analog', 0.018995030369961346),
   ('design', 0.015166574636480766),
   ('control', 0.014430333149272962),
   ('implement', 0.013215534695380084),
   ('neuron', 0.012184796613289159),
   ('signal', 0.010491441192711209),
   ('threshold', 0.010307380820909259)]),
 (18,
  [('code', 0.0350655248475412),
   ('representation', 0.02711820422992085),
   ('memory', 0.020825223822499028),
   ('bit', 0.019170883612300505),
   ('user', 0.013883482548332684),
   ('match', 0.011191124951342935),
   (

In [30]:
import numpy as np
def mallet_to_lda(mallet_model):
    model_gensim = gensim.models.ldamodel.LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = mallet_model.wordtopics
    return model_gensim

In [31]:
pyLDAvis.enable_notebook()
converted_mallet = mallet_to_lda(ldamallet)
vis = pyLDAvis.gensim_models.prepare(converted_mallet, corpus_all, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.105884  0.026037       1        1  6.449646
2     -0.162182 -0.095297       2        1  6.045128
15     0.244375 -0.099488       3        1  5.871600
4      0.018315  0.063591       4        1  5.727742
6      0.003687 -0.151744       5        1  5.588117
0     -0.189935 -0.017251       6        1  5.571457
12    -0.095112 -0.040680       7        1  5.438832
18    -0.003199  0.099137       8        1  5.254759
5      0.204018  0.093652       9        1  5.052010
16    -0.071430 -0.013813      10        1  4.975362
9     -0.052819 -0.129027      11        1  4.733061
3      0.037230  0.191522      12        1  4.714079
17     0.165761 -0.030477      13        1  4.577093
10     0.168917 -0.081892      14        1  4.542839
14    -0.020085  0.191131      15        1  4.431433
7     -0.212566 -0.072368      16        1  4.399847
11    -0.090676  0.049013      17        1  4.388028
1     -0.136437 -0.129034      18        1  4.310472
19    -0.095021  0.245528      19        1  4.164550
8      0.181274 -0.098538      20        1  3.763947, topic_info=            Term          Freq         Total Category  logprob  loglift
212        image  37285.000000  37285.000000  Default  30.0000  30.0000
586        layer  32859.000000  32859.000000  Default  29.0000  29.0000
926         hide  28251.000000  28251.000000  Default  28.0000  28.0000
511         cell  23435.000000  23435.000000  Default  27.0000  27.0000
385         rule  21123.000000  21123.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
334   processing   1531.192308  12885.610340  Topic20  -5.0762   1.1496
415        stage   1321.028658   7206.477802  Topic20  -5.2238   1.5831
976        range   1358.557881  11255.587783  Topic20  -5.1958   1.1653
1357        code   1358.557881  13364.002395  Topic20  -5.1958   0.9936
479       window   1088.347474   3893.826314  Topic20  -5.4176   2.0050

[1629 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5463      3  0.998842    abele
693       1  0.451113  ability
693       4  0.147384  ability
693      12  0.093627  ability
693      13  0.047710  ability
...     ...       ...      ...
856      16  0.081165    yield
856      17  0.022559    yield
856      18  0.201633    yield
856      19  0.022675    yield
856      20  0.046280    yield

[3457 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 3, 16, 5, 7, 1, 13, 19, 6, 17, 10, 4, 18, 11, 15, 8, 12, 2, 20, 9])

In [37]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_test_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.5022392405228553
